In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rtooling.apis.inference.anthropic import AnthropicModelBatch
from rtooling.data_models import ChatMessage, LLMResponse, MessageRole, Prompt
from rtooling.utils import utils

utils.setup_environment(logging_level="warning", openai_tag="OPENAI_API_KEY1", anthropic_tag="ANTHROPIC_API_KEY_BATCH")

In [ ]:
batch_model = AnthropicModelBatch()
prompts = [
    Prompt(messages=[ChatMessage(content="Say 1", role=MessageRole.user)]),
    Prompt(messages=[ChatMessage(content="Say 2", role=MessageRole.user)]),
    Prompt(messages=[ChatMessage(content="Say 3", role=MessageRole.user)]),
    Prompt(messages=[ChatMessage(content="Say 4", role=MessageRole.user)]),
    Prompt(messages=[ChatMessage(content="Say 5", role=MessageRole.user)]),
]

In [ ]:
batches = batch_model.list_message_batches()
batches


In [ ]:
requests = batch_model.prompts_to_requests(model_id="claude-3-5-sonnet-20240620", prompts=prompts, max_tokens=10)
print(requests)

In [ ]:
batch_response = batch_model.create_message_batch(requests=requests)
batch_id = batch_response.id

print(batch_id)
print(batch_response)

In [ ]:
batch_response = batch_model.retrieve_message_batch(batch_id)
print(batch_response)

In [ ]:
poll_response = await batch_model.poll_message_batch(batch_id)
results = batch_model.retrieve_message_batch_results(batch_id)

print(results)

In [ ]:
responses = []
for result in results:
    if result.result.type == "succeeded":
        content = result.result.message.content
        text = content[0].text if content else ""
        responses.append(
            LLMResponse(
                model_id="claude-3-5-sonnet-20240620",
                completion=text,
                stop_reason=result.result.message.stop_reason,
                duration=None,  # Batch does not track individual durations
                api_duration=None,
                cost=0,
            )
        )
responses

In [ ]:

responses, batch_id = await batch_model(model_ids=("claude-3-5-sonnet-20240620",), prompts=prompts[0:2], max_tokens=10)

print(responses)
print(batch_id)
